In [1]:

import os, sys, torch
print('Python:', sys.version)
print('PyTorch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('CUDA device:', torch.cuda.get_device_name(0))

# Check presence of mamba_ssm (optional) and mamba_model (required)
try:
    import importlib.util
    spec = importlib.util.find_spec('mamba_ssm')
    print('mamba_ssm found:', spec is not None)
except Exception as e:
    print('mamba_ssm check error:', e)

try:
    spec2 = importlib.util.find_spec('mamba_model')
    print('mamba_model found:', spec2 is not None)
except Exception as e:
    print('mamba_model check error:', e)


Python: 3.10.18 (main, Jun  5 2025, 13:14:17) [GCC 11.2.0]
PyTorch: 2.8.0+cu129
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4060 Ti
mamba_ssm found: True
mamba_model found: True


In [3]:
#imports
import os
import h5py
import numpy as np
import cv2
import torch
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn.functional as F
from torch.optim import AdamW


In [4]:
# Constants
PATCH_SIZE = 256
TARGET_SIZE = (256, 256)
LABEL_KEY = 'outlines'
MODALITIES_3 = ['dem', 'optical', 'bright_dark_outlines']
CHANNEL_INFO_3 = {'dem': 2, 'optical': 6, 'bright_dark_outlines': 3}

def normalize(arr):
    arr = arr.astype(np.float32)
    return (arr - arr.mean()) / (arr.std() + 1e-5)

In [5]:
# %%
import os, random, h5py
import numpy as np
import torch
from torch.utils.data import Dataset

# --- Normalization ---
def normalize(arr):
    arr = arr.astype(np.float32)
    return (arr - arr.mean()) / (arr.std() + 1e-5)

# --- Augmentations ---
def augment_patch(img, label):
    if random.random() < 0.5:
        img = np.flip(img, axis=0)
        label = np.flip(label, axis=0)
    if random.random() < 0.5:
        img = np.flip(img, axis=1)
        label = np.flip(label, axis=1)
    if random.random() < 0.5:
        noise = np.random.normal(0, 0.01, img.shape)
        img = img + noise
    return img, label

# --- Dataset Class ---
class GlacierHDF5PatchDataset3(Dataset):
    def __init__(self, hdf5_file_path, patch_size=256, target_size=(256,256),
                 length=600, modalities=None):
        self.hdf5 = h5py.File(hdf5_file_path, 'r')
        self.modalities = modalities if modalities is not None else ['dem', 'optical', 'bright_dark_outlines']
        self.channel_info = {'dem': 2, 'optical': 6, 'bright_dark_outlines': 3}
        self.in_chans = sum(self.channel_info[m] for m in self.modalities)
        self.tiles = [name for name in self.hdf5.keys() if all(m in self.hdf5[name] for m in self.modalities)]
        self.patch_size = patch_size
        self.target_size = target_size
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        tile_name = random.choice(self.tiles)
        tile = self.hdf5[tile_name]
        h, w = tile[self.modalities[0]].shape[:2]
        y = random.randint(0, h - self.patch_size)
        x = random.randint(0, w - self.patch_size)

        input_channels = []
        for key in self.modalities:
            arr = tile[key][y:y+self.patch_size, x:x+self.patch_size, :]
            arr = normalize(arr)
            input_channels.append(arr)
        input_patch = np.concatenate(input_channels, axis=2)

        label = tile['outlines'][y:y+self.patch_size, x:x+self.patch_size]
        if label.ndim == 3:
            label = label[:,:,0] if label.shape[2] == 1 else np.argmax(label, axis=2)

        input_patch, label = augment_patch(input_patch, label)

        input_tensor = torch.tensor(np.ascontiguousarray(input_patch)).permute(2,0,1).float()
        label_tensor = torch.tensor(np.ascontiguousarray(label), dtype=torch.long)
        return input_tensor, label_tensor

    def close(self):
        self.hdf5.close()

# --- Dataset Ablation Variants ---
variants = [
    {"name": "full_2000",        "modalities": ['dem', 'optical', 'bright_dark_outlines'], "length_train": 2000, "length_val": 500},
    {"name": "no_dem_1000",      "modalities": ['optical', 'bright_dark_outlines'],        "length_train": 1000, "length_val": 250},
    {"name": "no_optical_1000",  "modalities": ['dem', 'bright_dark_outlines'],            "length_train": 1000, "length_val": 250},
    {"name": "no_outlines_2000", "modalities": ['dem', 'optical'],                         "length_train": 2000, "length_val": 500},
    {"name": "optical_only_500", "modalities": ['optical'],                                "length_train": 500,  "length_val": 125},
    {"name": "full_500",         "modalities": ['dem', 'optical', 'bright_dark_outlines'], "length_train": 500,  "length_val": 125},
]


In [6]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    jaccard_score, 
    precision_score, 
    recall_score, 
    confusion_matrix
)

def segmentation_metrics(y_true, y_pred, num_classes=3):
    # Flatten arrays
    y_true = y_true.ravel()
    y_pred = y_pred.ravel()

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))

    # Pixel accuracy
    pixel_acc = accuracy_score(y_true, y_pred)

    # IoU (Jaccard Index)
    per_class_iou = jaccard_score(y_true, y_pred, average=None, labels=list(range(num_classes)))
    mean_iou = jaccard_score(y_true, y_pred, average="macro", labels=list(range(num_classes)))

    # Dice (F1-score)
    per_class_dice = f1_score(y_true, y_pred, average=None, labels=list(range(num_classes)))
    mean_dice = f1_score(y_true, y_pred, average="macro", labels=list(range(num_classes)))

    # Precision & Recall
    precision = precision_score(y_true, y_pred, average="macro", labels=list(range(num_classes)))
    recall = recall_score(y_true, y_pred, average="macro", labels=list(range(num_classes)))

    return {
        "pixel_acc": pixel_acc,
        "mean_iou": mean_iou,
        "mean_dice": mean_dice,
        "per_class_iou": per_class_iou,
        "per_class_dice": per_class_dice,
        "precision": precision,
        "recall": recall,
        "confusion_matrix": cm
    }


In [7]:
class EarlyStopping:
    """
    Early-stop when monitored metric doesn't improve after `patience` epochs.
    mode='max' for metrics like mIoU; set delta to require minimal improvement.
    """
    def __init__(self, patience=10, mode='max', min_delta=0.0):
        self.patience = patience
        self.mode = mode
        self.min_delta = min_delta
        self.best = None
        self.counter = 0
        self.should_stop = False

    def step(self, current):
        if self.best is None:
            self.best = current
            return False

        improved = (current > self.best + self.min_delta) if self.mode == 'max' else (current < self.best - self.min_delta)
        if improved:
            self.best = current
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        return self.should_stop


In [8]:
import torch.optim as optim

def build_optimizer(model, opt_cfg=None):
    opt_cfg = opt_cfg or {"name": "AdamW", "lr": 3e-6, "weight_decay": 1e-4}
    name = opt_cfg.get("name", "AdamW").lower()
    lr = opt_cfg.get("lr", 3e-6)
    wd = opt_cfg.get("weight_decay", 1e-4)
    momentum = opt_cfg.get("momentum", 0.9)

    if name == "sgd":
        return optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=momentum, nesterov=True)
    elif name == "adam":
        return optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    else:
        return optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

def build_scheduler(optimizer, sch_cfg=None):
    """
    Default: ReduceLROnPlateau on val mIoU (mode='max').
    If you want cosine/step later, we can expand this switch.
    """
    sch_cfg = sch_cfg or {"name": "ReduceLROnPlateau", "factor": 0.5, "patience": 5, "threshold": 1e-4, "min_lr": 1e-7}
    name = sch_cfg.get("name", "ReduceLROnPlateau").lower()

    if name == "reducelronplateau":
        return optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="max",
            factor=sch_cfg.get("factor", 0.5),
            patience=sch_cfg.get("patience", 5),
            threshold=sch_cfg.get("threshold", 1e-4),
            cooldown=sch_cfg.get("cooldown", 0),
            min_lr=sch_cfg.get("min_lr", 1e-7)
        )
    elif name == "cosineannealing":
        return optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=sch_cfg.get("T_max", 50), eta_min=sch_cfg.get("min_lr", 1e-7)
        )
    elif name == "steplr":
        return optim.lr_scheduler.StepLR(
            optimizer, step_size=sch_cfg.get("step_size", 30), gamma=sch_cfg.get("gamma", 0.1)
        )
    else:
        return None


In [9]:
from tqdm import tqdm
def train_epoch(loader, model, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for x, y in tqdm(loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss/len(loader)

def eval_epoch(loader, model, criterion, device):
    model.eval()
    total_loss = 0
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y).sum().item()
            total += y.numel()
    return total_loss/len(loader), correct/total


In [10]:
TRAIN_PATH = "/home/goblin/dataset/20230905_train_global_ps384.hdf5"
VAL_PATH = "/home/goblin/dataset/20230905_val_global_ps384.hdf5"
TEST_PATH = "/home/goblin/dataset/20230905_test_global_ps384.hdf5"
WEIGHT_DIR = "weights/NIRD_Config"
os.makedirs(WEIGHT_DIR, exist_ok=True)

# dataset_train = GlacierHDF5PatchDataset3(TRAIN_PATH, length=2000)
# dataset_val = GlacierHDF5PatchDataset3(VAL_PATH, length = 500)
# dataset_test = GlacierHDF5PatchDataset3(TEST_PATH, length = 800)
# train_loader= DataLoader(dataset_train, batch_size=30, shuffle=True)
# dev_loader = DataLoader(dataset_val, batch_size=30, shuffle=False)
# dev_loader = DataLoader(dataset_test, batch_size=30, shuffle=False)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [11]:
import torch
import time
from thop import profile
from torchinfo import summary

def analyze_model(model, input_size=(1, 11, 256, 256), device=None):
    # --- Device selection ---
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[INFO] Using device: {device}")

    # --- Move model and input to device ---
    model = model.to(device)
    dummy_input = torch.randn(*input_size).to(device)

    # --- Verify device consistency ---
    if next(model.parameters()).device != dummy_input.device:
        raise RuntimeError(f"Model and input device mismatch: model on {next(model.parameters()).device}, input on {dummy_input.device}")

    # --- FLOPs & MACs ---
    try:
        macs, params = profile(model, inputs=(dummy_input,), verbose=False)
        flops = 2 * macs  # 1 MAC = 2 FLOPs
    except Exception as e:
        print(f"[ERROR] thop profiling failed: {e}")
        macs, params, flops = 0, 0, 0  # Fallback values

    # --- Torchinfo summary ---
    try:
        print(summary(model, input_size=input_size, device=device))
    except Exception as e:
        print(f"[ERROR] torchinfo summary failed: {e}")

    # --- Inference time ---
    model.eval()
    with torch.no_grad():
        # Warm-up
        for _ in range(5):
            _ = model(dummy_input)
        torch.cuda.synchronize() if device.startswith("cuda") else None  # Sync for GPU
        start = time.time()
        for _ in range(20):
            _ = model(dummy_input)
        torch.cuda.synchronize() if device.startswith("cuda") else None  # Sync for GPU
        end = time.time()
    avg_time = (end - start) / 20 * 1000  # Convert to ms/sample

    return {
        "device": device,
        "params": params,
        "macs": macs / 1e6,  # Convert to millions
        "flops": flops / 1e6,  # Convert to millions
        "avg_inference_time_ms": avg_time
    }

# Example usage
try:
    model = SegFormer(num_classes=2, in_chans=11)  # Ensure SegFormer is defined
    stats = analyze_model(model, input_size=(1, 11, 256, 256), device="cpu")
    print(stats)
except Exception as e:
    print(f"[ERROR] Analysis failed: {e}")

[INFO] Using device: cpu
[ERROR] thop profiling failed: Pointer argument (at 0) cannot be accessed from Triton (cpu tensor?)
[ERROR] torchinfo summary failed: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [OverlapPatchEmbedInvo: 4, Involution2D: 5, AvgPool2d: 6, Conv2d: 6, ReLU: 6, Conv2d: 6, Conv2d: 5, BatchNorm2d: 5]
[ERROR] Analysis failed: Pointer argument (at 0) cannot be accessed from Triton (cpu tensor?)


In [12]:
# %%
import os, time, numpy as np, torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn

def _format_list(vals):
    return "[" + "|".join(f"{float(v):.4f}" for v in vals) + "]"

def _flatten_cm(cm):
    return "[" + "|".join(str(int(x)) for x in cm.flatten().tolist()) + "]"

def train_dataset_config(
    config,
    train_path,
    val_path,
    device="cuda",
    epochs_default=30,
    batch_size_default=30,
    num_classes=2,
    base_run_dir="runs",
    base_weight_dir="weights",
    save_every=5,
    es_patience_default=10
):
    name = config['name']
    run_dir = os.path.join(base_run_dir, name)
    weight_dir = os.path.join(base_weight_dir, name)
    os.makedirs(run_dir, exist_ok=True)
    os.makedirs(weight_dir, exist_ok=True)

    # Build datasets
    dataset_train = GlacierHDF5PatchDataset3(train_path, length=config["length_train"], modalities=config["modalities"])
    dataset_val   = GlacierHDF5PatchDataset3(val_path,   length=config["length_val"],   modalities=config["modalities"])

    epochs = config.get("epochs", epochs_default)
    batch_size = config.get("batch_size", batch_size_default)

    train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(dataset_val,   batch_size=batch_size, shuffle=False)

    # Build model
    in_chans = dataset_train.in_chans
    model = SegFormer(
        num_classes=num_classes,
        variant="mit_b0",
        drop_path_rate=0.1,
        in_chans=in_chans,
        embed_dims=[16, 32, 64, 128],  # keep fixed backbone for ablation
        depths=[1, 1, 1, 1],
        use_invo_stages=(True, True, False, False)
    ).to(device)

    # Optimizer & scheduler
    optimizer = build_optimizer(model, config.get("optimizer"))
    scheduler = build_scheduler(optimizer, config.get("scheduler"))

    # Criterion
    criterion = nn.CrossEntropyLoss()

    # TensorBoard writer
    writer = SummaryWriter(run_dir)

    # Optional: computational analysis
    try:
        computational_metrics = analyze_model(model, input_size=(1, in_chans, 256, 256), device=device)
    except Exception as e:
        print(f"[WARN] analyze_model failed: {e}")
        computational_metrics = {}
    if computational_metrics:
        if 'params' in computational_metrics:       writer.add_scalar("Computational/Params", computational_metrics['params'], 0)
        if 'macs' in computational_metrics:         writer.add_scalar("Computational/MACs_M", computational_metrics['macs'], 0)
        if 'flops' in computational_metrics:        writer.add_scalar("Computational/FLOPs_M", computational_metrics['flops'], 0)
        if 'avg_inference_time_ms' in computational_metrics:
            writer.add_scalar("Computational/AvgInference_ms", computational_metrics['avg_inference_time_ms'], 0)

    # Text log
    log_path = os.path.join(weight_dir, "training_log.txt")
    if not os.path.exists(log_path):
        with open(log_path, "w") as f:
            f.write("epoch,train_loss,val_loss,val_pixel_acc,val_mIoU,val_dice,val_precision,val_recall,lr,per_class_iou[],per_class_dice[],confusion_matrix_flat\n")

    # Early stopping
    es_cfg = config.get("early_stopping", {"monitor": "val_mIoU", "patience": es_patience_default})
    es_patience = es_cfg.get("patience", es_patience_default)
    early_stopper = EarlyStopping(patience=es_patience, mode="max", min_delta=0.0)

    best_iou, best_epoch = -1.0, -1
    start_time = time.time()

    for epoch in range(1, epochs + 1):
        # --- Train ---
        train_loss = train_epoch(train_loader, model, criterion, optimizer, device)

        # --- Validate ---
        val_loss, _ = eval_epoch(val_loader, model, criterion, device)

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                preds  = torch.argmax(logits, dim=1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        y_true = np.concatenate(all_labels, axis=None)
        y_pred = np.concatenate(all_preds, axis=None)
        metrics = segmentation_metrics(y_true, y_pred, num_classes=num_classes)

        val_acc   = float(metrics["pixel_acc"])
        val_miou  = float(metrics["mean_iou"])
        val_dice  = float(metrics["mean_dice"])
        val_prec  = float(metrics["precision"])
        val_rec   = float(metrics["recall"])
        cls_iou   = metrics["per_class_iou"]
        cls_dice  = metrics["per_class_dice"]
        cm        = metrics["confusion_matrix"]

        # --- LR & Scheduler ---
        current_lr = optimizer.param_groups[0]["lr"]
        if scheduler is not None:
            scheduler.step(val_miou)
            current_lr = optimizer.param_groups[0]["lr"]

        # --- TensorBoard ---
        writer.add_scalar("Loss/train",    train_loss, epoch)
        writer.add_scalar("Loss/val",      val_loss,   epoch)
        writer.add_scalar("Val/PixelAcc",  val_acc,    epoch)
        writer.add_scalar("Val/mIoU",      val_miou,   epoch)
        writer.add_scalar("Val/Dice",      val_dice,   epoch)
        writer.add_scalar("Val/Precision", val_prec,   epoch)
        writer.add_scalar("Val/Recall",    val_rec,    epoch)
        writer.add_scalar("LR/lr",         current_lr, epoch)

        # --- Text log (CSV line) ---
        with open(log_path, "a") as f:
            f.write(f"{epoch},{train_loss:.6f},{val_loss:.6f},{val_acc:.6f},{val_miou:.6f},{val_dice:.6f},{val_prec:.6f},{val_rec:.6f},{current_lr:.8e},{_format_list(cls_iou)},{_format_list(cls_dice)},{_flatten_cm(cm)}\n")

        # --- Checkpointing ---
        if val_miou > best_iou:
            best_iou, best_epoch = val_miou, epoch
            best_ckpt_path = os.path.join(weight_dir, "best_model.pth")
            torch.save(model.state_dict(), best_ckpt_path)
            print(f"[{name}] ✔ New best mIoU={best_iou:.4f} at epoch {epoch} → saved {best_ckpt_path}")

        if save_every is not None and epoch % save_every == 0:
            ep_path = os.path.join(weight_dir, f"epoch_{epoch:03d}.pth")
            torch.save(model.state_dict(), ep_path)

        # --- Early stopping ---
        if early_stopper.step(val_miou):
            print(f"[{name}] ⏹ Early stopping at epoch {epoch} (no improvement for {es_patience} epochs).")
            break

    total_time = time.time() - start_time
    with open(log_path, "a") as f:
        f.write(f"BEST_EPOCH={best_epoch}\nBEST_mIoU={best_iou:.6f}\nTOTAL_TIME_SEC={total_time:.2f}\n")

    writer.flush()
    writer.close()

    dataset_train.close()
    dataset_val.close()

    print(f"[{name}] Finished. Best mIoU={best_iou:.4f} at epoch {best_epoch}. Log: {log_path}")


In [13]:
# %%
def run_all_dataset_configs(
    variants,
    train_path,
    val_path,
    device="cuda",
    epochs=30,
    batch_size=30,
    num_classes=2,
    base_run_dir="runs",
    base_weight_dir="weights"
):
    """
    Loop through all dataset configs (variants) and train each one.
    """
    for config in variants:
        print(f"\n=== Processing dataset config: {config['name']} ===")
        
        train_dataset_config(
            config=config,
            train_path=train_path,
            val_path=val_path,
            device=device,
            epochs_default=epochs,
            batch_size_default=batch_size,
            num_classes=num_classes,
            base_run_dir=base_run_dir,
            base_weight_dir=base_weight_dir,
            save_every=config.get("save_every", None),
            es_patience_default=config.get("early_stopping", {"patience": 10})["patience"]
        )


In [14]:
import pandas as pd

def collect_dataset_results(variants, base_weight_dir="weights/ablation", out_dir="results"):
    """
    Collect BEST_EPOCH, BEST_mIoU, TOTAL_TIME_SEC from all dataset training logs.
    Save as CSV + Markdown.
    """
    os.makedirs(out_dir, exist_ok=True)
    results = []

    for cfg in variants:
        name = "dataset_" + cfg["name"]
        log_path = os.path.join(base_weight_dir, name, "training_log.txt")
        if not os.path.exists(log_path):
            print(f"[WARN] No log file found for {name}, skipping.")
            continue

        best_epoch, best_miou, total_time = None, None, None
        with open(log_path, "r") as f:
            for line in f:
                if line.startswith("BEST_EPOCH="):
                    best_epoch = int(line.strip().split("=")[1])
                elif line.startswith("BEST_mIoU="):
                    best_miou = float(line.strip().split("=")[1])
                elif line.startswith("TOTAL_TIME_SEC="):
                    total_time = float(line.strip().split("=")[1])

        results.append({
            "Config": name,
            "BEST_EPOCH": best_epoch,
            "BEST_mIoU": best_miou,
            "TOTAL_TIME_SEC": total_time
        })

    # Save results
    df = pd.DataFrame(results)
    csv_path = os.path.join(out_dir, "dataset_ablation_results.csv")
    md_path = os.path.join(out_dir, "dataset_ablation_results.md")

    df.to_csv(csv_path, index=False)
    df.to_markdown(md_path, index=False)

    print(f"[INFO] Saved dataset ablation results → {csv_path}, {md_path}")
    return df


In [15]:
# Example usage:
variants = [
    {"name": "full_2000", "modalities": ['dem', 'optical', 'bright_dark_outlines'], "length_train": 2000, "length_val": 500},
    {"name": "no_dem_1000", "modalities": ['optical', 'bright_dark_outlines'], "length_train": 1000, "length_val": 250},
    {"name": "no_optical_1000", "modalities": ['dem', 'bright_dark_outlines'], "length_train": 1000, "length_val": 250},
    {"name": "no_outlines_2000", "modalities": ['dem', 'optical'], "length_train": 2000, "length_val": 500},
    {"name": "optical_only_500", "modalities": ['optical'], "length_train": 500, "length_val": 125},
    {"name": "full_500", "modalities": ['dem', 'optical', 'bright_dark_outlines'], "length_train": 500, "length_val": 125},
]

# Run all dataset configs
run_all_dataset_configs(
    variants=variants,
    train_path=TRAIN_PATH,
    val_path=VAL_PATH,
    device=DEVICE,
    epochs=30,
    batch_size=30,
    num_classes=2,
    base_run_dir="runs/ablation",
    base_weight_dir="weights/ablation"
)

# Collect summary table
df = collect_dataset_results(variants, base_weight_dir="weights/ablation", out_dir="results")
print(df)



=== Processing dataset config: full_2000 ===
[INFO] Using device: cuda
Layer (type:depth-idx)                             Output Shape              Param #
SegFormer                                          [1, 2, 256, 256]          --
├─MixVisionTransformer: 1-1                        [1, 16, 128, 128]         --
│    └─ModuleList: 2-1                             --                        --
│    │    └─MiTStage: 3-1                          [1, 16, 128, 128]         5,795
│    │    └─MiTStage: 3-2                          [1, 32, 64, 64]           19,057
│    │    └─MiTStage: 3-3                          [1, 64, 32, 32]           84,416
│    │    └─MiTStage: 3-4                          [1, 128, 16, 16]          322,432
├─SegFormerHead: 1-2                               [1, 2, 128, 128]          --
│    └─ModuleList: 2-2                             --                        --
│    │    └─Sequential: 3-5                        [1, 128, 128, 128]        2,304
│    │    └─Sequential: 

100%|██████████| 67/67 [01:49<00:00,  1.64s/it]


[full_2000] ✔ New best mIoU=0.6791 at epoch 1 → saved weights/ablation/full_2000/best_model.pth


100%|██████████| 67/67 [01:55<00:00,  1.72s/it]


[full_2000] ✔ New best mIoU=0.7607 at epoch 2 → saved weights/ablation/full_2000/best_model.pth


...

[full_500] ✔ New best mIoU=0.7149 at epoch 20 → saved weights/ablation/full_500/best_model.pth


100%|██████████| 17/17 [00:25<00:00,  1.52s/it]


[full_500] ✔ New best mIoU=0.7445 at epoch 24 → saved weights/ablation/full_500/best_model.pth


100%|██████████| 17/17 [00:26<00:00,  1.55s/it]


[full_500] Finished. Best mIoU=0.7445 at epoch 24. Log: weights/ablation/full_500/training_log.txt
[WARN] No log file found for dataset_full_2000, skipping.
[WARN] No log file found for dataset_no_dem_1000, skipping.
[WARN] No log file found for dataset_no_optical_1000, skipping.
[WARN] No log file found for dataset_no_outlines_2000, skipping.
[WARN] No log file found for dataset_optical_only_500, skipping.
[WARN] No log file found for dataset_full_500, skipping.
[INFO] Saved dataset ablation results → results/dataset_ablation_results.csv, results/dataset_ablation_results.md
Empty DataFrame
Columns: []
Index: []


In [1]:
# %%
import os, pandas as pd, matplotlib.pyplot as plt

def parse_training_log(log_path):
    """
    Parse a training_log.txt file into a DataFrame (epochs) and best metrics dict.
    """
    df = []
    best = {"Config": os.path.basename(os.path.dirname(log_path))}
    with open(log_path, "r") as f:
        lines = f.readlines()

    # Separate header/data vs best lines
    header = None
    for line in lines:
        line = line.strip()
        if line.startswith("epoch,"):
            header = line.split(",")
        elif line.startswith("BEST_EPOCH") or line.startswith("BEST_mIoU") or line.startswith("TOTAL_TIME_SEC"):
            key, val = line.split("=")
            best[key.strip()] = val.strip()
        elif header and line[0].isdigit():
            parts = line.split(",")
            row = dict(zip(header, parts))
            df.append(row)

    df = pd.DataFrame(df)
    df = df.apply(pd.to_numeric, errors="ignore")
    return df, best


def collect_all_logs(base_dir="weights/ablation"):
    """
    Collect all training logs from dataset ablation runs.
    """
    all_dfs, all_bests = {}, []
    for config in os.listdir(base_dir):
        log_path = os.path.join(base_dir, config, "training_log.txt")
        if not os.path.exists(log_path):
            continue
        df, best = parse_training_log(log_path)
        all_dfs[config] = df
        best["Config"] = config
        all_bests.append(best)
    return all_dfs, pd.DataFrame(all_bests)


def plot_metric(all_dfs, metric, out_dir):
    """
    Plot one metric (line plot with all configs overlapped).
    """
    plt.figure(figsize=(8, 5))
    for config, df in all_dfs.items():
        if metric in df.columns:
            plt.plot(df["epoch"], df[metric], label=config)
    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.title(f"{metric} over epochs")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{metric}_lineplot.png"))
    plt.close()


def plot_bar(df_summary, metric, out_dir):
    """
    Bar plot of best metrics per config.
    """
    plt.figure(figsize=(8, 5))
    plt.bar(df_summary["Config"], df_summary[metric].astype(float))
    plt.ylabel(metric)
    plt.title(f"Best {metric} per config")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{metric}_barplot.png"))
    plt.close()


# === Main workflow ===
base_dir = "weights/ablation"
out_dir = "results/plots/dataset_ablation_study"
os.makedirs(out_dir, exist_ok=True)

# Collect logs
all_dfs, df_summary = collect_all_logs(base_dir)

# Save summary as CSV + Markdown
csv_path = os.path.join(out_dir, "summary.csv")
md_path  = os.path.join(out_dir, "summary.md")
df_summary.to_csv(csv_path, index=False)

with open(md_path, "w") as f:
    f.write(df_summary.to_markdown(index=False))

print(f"[INFO] Saved summary → {csv_path}, {md_path}")

# Plot epoch-wise metrics
metrics_to_plot = ["train_loss", "val_loss", "val_pixel_acc", "val_mIoU", "val_dice", "val_precision", "val_recall"]
for m in metrics_to_plot:
    plot_metric(all_dfs, m, out_dir)

# Plot bar charts for best values
for m in ["BEST_mIoU", "BEST_EPOCH", "TOTAL_TIME_SEC"]:
    if m in df_summary.columns:
        plot_bar(df_summary, m, out_dir)

print(f"[INFO] Plots saved to {out_dir}")


/tmp/ipykernel_7367/540894653.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors="ignore")


[INFO] Saved summary → results/plots/dataset_ablation_study/summary.csv, results/plots/dataset_ablation_study/summary.md
[INFO] Plots saved to results/plots/dataset_ablation_study
